<a href="https://colab.research.google.com/github/ManojBhuma/WebscrapingAmazon/blob/main/Webscraping_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

In [ ]:
itemName = input("Enter Item Name: ")
URL = "https://www.amazon.ca/s?k="+itemName
webpage = requests.get(URL, headers=HEADERS)

Enter Item Name: amazon fire stick


In [ ]:
soup = BeautifulSoup(webpage.content, "lxml")
link = soup.findAll('a',attrs={'class':'a-link-normal a-text-normal'})
#Get First Product from search
productPage = "https://www.amazon.ca"+link[0]['href']

In [ ]:
productPage = requests.get(productPage, headers=HEADERS)

In [ ]:
soup2 = BeautifulSoup(productPage.content, "lxml")

In [ ]:
title = soup2.find("span", attrs={"id":'productTitle'})

In [ ]:
productTitle = title.string.strip()

In [ ]:
print(productTitle)

Fire TV Stick (3rd Gen) with Alexa Voice Remote (includes TV controls) | HD streaming device | 2021 release


In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
#Scrape Product Details
#{"id":'feature-bullets'}
def get_item_Description(soup):
    description = soup.find("span", attrs={"id":'productTitle'})
    descriptionText = description.string.strip()
    return descriptionText

def get_itemPrice(soup):
    price = soup.find("span", attrs={"id":'price_inside_buybox'})
    priceText = price.string.strip()
    return priceText

def get_vendor(soup):
    try:
        vendor = soup.find("a", attrs={"id":'sellerProfileTriggerId'})
        vendorText = vendor.string.strip()
        return vendorText
    except AttributeError:
        return "Sold By Amazon"
    

In [ ]:
itemDescription = get_item_Description(soup2)
print(itemDescription)

Fire TV Stick (3rd Gen) with Alexa Voice Remote (includes TV controls) | HD streaming device | 2021 release


In [ ]:
itemPrice = get_itemPrice(soup2)
print(itemPrice)

$59.99


In [ ]:
vendorName = get_vendor(soup2)
print(vendorName)

Sold By Amazon


In [ ]:
links = []
for i in soup2.findAll("a",{'data-hook':"see-all-reviews-link-foot"}):
    links.append(i['href'])

In [ ]:
def searchReviews(reviewLink):
    url="https://www.amazon.ca"+reviewLink
    page=requests.get(url,headers=HEADERS)
    if page.status_code==200:
        return page
    else:
        return "Error"

In [ ]:
reviews=[]
reviewerName=[]
ratings=[]
reviewDate=[]
for j in range(len(links)):
    for k in range(50):
        response=searchReviews(links[j]+'&pageNumber='+str(k+1))
        soup=BeautifulSoup(response.content)
        for i in soup.findAll("span",{'data-hook':"review-body"}):
            reviews.append(i.text.strip())
        for i in soup.findAll("span",{'class':"a-profile-name"})[2:]:
            reviewerName.append(i.text.strip())
        for i in soup.findAll("i",{'data-hook':"review-star-rating"}):
            ratings.append(i.text.strip())
        for i in soup.findAll("span",{'data-hook':"review-date"}):
            reviewDate.append(i.text.strip()) 

In [ ]:
import pandas as pd

In [ ]:
itemD = {'itemName':[itemName],'itemDescription':[itemDescription],'itemPrice':[itemPrice],'vendorName':[vendorName]}
items_Data = pd.DataFrame.from_dict(itemD)
items_Data.to_csv('items.csv',index=False)

In [ ]:
print(len(reviewerName))
print(len(reviews))
print(len(ratings))
print(len(reviewDate))

119
119
119
119


In [ ]:
dicti = {'reviewerName':reviewerName,'reviews':reviews,'ratings':ratings,'reviewDate':reviewDate}
reviews_Data = pd.DataFrame.from_dict(dicti)
reviews_Data.to_csv('reviews.csv',index=False)